# Test registration on velodyne scans

In [ ]:
%matplotlib widget
import numpy as np
import os
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from copy import deepcopy
import open3d as o3d

import planeslam.io as io
from planeslam.scan import velo_pc_to_scan
from planeslam.general import NED_to_ENU, trajectory_plot_trace, pc_plot_trace
from planeslam.geometry.util import quat_to_R
from planeslam.registration import get_correspondences
from planeslam.point_cloud import velo_preprocess

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
# Read in point cloud data
pcpath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_12_2022', 'flightroom', 'run_1', 'pcs')
PCs = []
#len(os.listdir(pcpath))
select_idxs = np.arange(0, 1000, 2)
for i in select_idxs:  
    filename = pcpath+'\pc_'+str(i)+'.npy'
    PC = np.load(filename)
    PCs.append(PC)

In [ ]:
# Read in pose data
posepath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_12_2022', 'flightroom', 'run_1', 'poses')
poses = []
for i in select_idxs:  
    filename = posepath+'\pose_'+str(i)+'.npy'
    pose = np.load(filename)
    poses.append(pose)

In [ ]:
rover_rotations = np.zeros((3,3,len(poses)))
for i in range(len(poses)):
    rover_rotations[:,:,i] = quat_to_R(poses[i][3:])

In [ ]:
# Plot ground-truth trajectory
rover_positions = np.asarray(poses)[:,:3]
gt_traj_trace = go.Scatter3d(x=rover_positions[:,0], y=rover_positions[:,1], z=rover_positions[:,2], 
    marker=dict(size=5), hovertext=np.arange(len(rover_positions)))
gt_traj_trace.name = "Ground-truth"
fig = go.Figure(data=gt_traj_trace)
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Extract scans
num_scans = len(PCs)
scans = num_scans * [None]
# scans_transformed = num_scans * [None]
avg_extraction_time = 0
for i in range(num_scans):
    #print(i)
    start_time = time.time()
    P = velo_preprocess(PCs[i], poses[i])
    scans[i] = velo_pc_to_scan(P)
    scans[i].remove_small_planes(area_thresh=0.1)
    scans[i].reduce_inside(p2p_dist_thresh=0.1)
    #scans[i].transform(quat_to_R(poses[s][3:]), poses[s][:3])
    avg_extraction_time += time.time() - start_time
avg_extraction_time /= num_scans
print(avg_extraction_time)

In [ ]:
# Plot 2 scans
source_idx = 126
target_idx = 26
source = scans[source_idx]
target = scans[target_idx]

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

for t in source.plot_trace(show_normals=False):
    fig.add_trace(t, row=1, col=1)
#fig.add_trace(pc_plot_trace(PCs[source_idx]), row=1, col=1)

for t in target.plot_trace(show_normals=False):
    fig.add_trace(t, row=1, col=2)
#fig.add_trace(pc_plot_trace(PCs[target_idx]), row=1, col=2)

fig.update_layout(width=1600, height=700, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
fig.show()

correspondences = get_correspondences(source, target)
print(correspondences)

In [ ]:
from planeslam.registration import robust_GN_register

R_hat, t_hat = robust_GN_register(source, target, t_loss_thresh=0.1)

print(R_hat)
print(t_hat)

In [ ]:
from planeslam.geometry.util import quat_to_R

R_1 = quat_to_R(poses[target_idx][3:])
R_2 = quat_to_R(poses[source_idx][3:])
R_gt = R_2 @ R_1.T

t_gt = poses[source_idx][:3] - poses[target_idx][:3]
print(R_gt)
print(t_gt)

In [ ]:
from planeslam.registration import robust_GN_register

R_abs = quat_to_R(poses[0][3:])
t_abs = poses[0][:3].copy()
traj_est = np.zeros((len(scans), 3))
traj_est[0] = t_abs
traj_Rs = np.zeros((3, 3, num_scans))
traj_Rs[:,:,0] = R_abs
avg_runtime = 0

R_hats = []
t_hats = []

for i in range(1, len(scans)):
    print("i = ", i)
    start_time = time.time()
    R_hat, t_hat = robust_GN_register(scans[i], scans[i-1], t_loss_thresh=0.1)
    t_abs += (R_abs @ t_hat).flatten()
    R_abs = R_hat @ R_abs
    avg_runtime += time.time() - start_time
    traj_est[i] = t_abs
    traj_Rs[:,:,i] = R_abs

    R_hats.append(R_hat)
    t_hats.append(t_hat)

    R_1 = quat_to_R(poses[i-1][3:])
    R_2 = quat_to_R(poses[i][3:])
    R_gt = R_2 @ R_1.T

    t_gt = poses[i][:3] - poses[i-1][:3]
    # print("R_hat: ", R_hat)
    # print("R_gt: ", R_gt)
    # print("t_hat: ", t_hat.flatten())
    # print("t_gt: ", t_gt)

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

In [ ]:
# Plot trajectories
est_traj_trace = go.Scatter3d(x=traj_est[:,0], y=traj_est[:,1], z=traj_est[:,2], hovertext=np.arange(len(traj_est)), marker=dict(size=5))
fig = go.Figure(data=[gt_traj_trace, est_traj_trace])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

Add loop closures

In [ ]:
from graphslam.graph import Graph
from graphslam.vertex import Vertex
from graphslam.edge.edge_odometry import EdgeOdometry
from graphslam.pose.se3 import PoseSE3

from planeslam.geometry.util import R_to_quat

In [ ]:
vertices = []
edges = []

# Add first vertex
p = PoseSE3(traj_est[0], R_to_quat(traj_Rs[:,:,0]))
v = Vertex(0, p)
vertices.append(v)

# For each scan
for i in range(1, num_scans):
    # Add new vertex
    p = PoseSE3(traj_est[i], R_to_quat(traj_Rs[:,:,i]))
    v = Vertex(i, p)
    vertices.append(v)

    # Add odometry edge
    information = np.eye(6)
    estimate = PoseSE3(t_hats[i-1], R_to_quat(R_hats[i-1]))
    estimate.normalize()
    e = EdgeOdometry([i-1, i], information, estimate)
    edges.append(e)

In [ ]:
source_idx = 126
target_idx = 26
source = scans[source_idx]
target = scans[target_idx]

source_transformed = deepcopy(source)
source_transformed.transform(traj_Rs[:,:,source_idx], traj_est[source_idx])
target_transformed = deepcopy(target)
target_transformed.transform(traj_Rs[:,:,target_idx], traj_est[target_idx])

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

for t in source_transformed.plot_trace(show_normals=False):
    fig.add_trace(t, row=1, col=1)
#fig.add_trace(pc_plot_trace(PCs[source_idx]), row=1, col=1)

for t in target_transformed.plot_trace(show_normals=False):
    fig.add_trace(t, row=1, col=2)
#fig.add_trace(pc_plot_trace(PCs[target_idx]), row=1, col=2)

fig.update_layout(width=1600, height=700, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
fig.show()

correspondences = get_correspondences(source_transformed, target_transformed)
print(correspondences)

In [ ]:
from planeslam.registration import decoupled_GN_opt

R_hat, t_hat, t_loss, _ = decoupled_GN_opt(source, target, correspondences)
print(R_hat)
print(t_hat)

In [ ]:
from planeslam.registration import loop_closure_register

# Add loop closure edges
loop_closures = [(26, 126), (54, 155)]#, (84, 244)]

for (i,j) in loop_closures:
    R_hat, t_hat = loop_closure_register(scans[j], scans[i], (traj_Rs[:,:,j], traj_est[j]), (traj_Rs[:,:,i], traj_est[i]), t_loss_thresh=0.1)
    information = np.eye(6)
    estimate = PoseSE3(t_hat, R_to_quat(R_hat))
    estimate.normalize()
    e = EdgeOdometry([i, j], information, estimate)
    edges.append(e)

In [ ]:
g = Graph(edges, vertices)
g.optimize()

In [ ]:
positions = np.zeros((num_scans, 3))
for i, v in enumerate(g._vertices):
    positions[i] = v.pose.position

rotations = np.zeros((3, 3, num_scans))
for i, v in enumerate(g._vertices):
    rotations[:,:,i] = quat_to_R(v.pose.orientation)

In [ ]:
est_traj_trace = go.Scatter3d(x=positions[:,0], y=positions[:,1], z=positions[:,2], 
    marker=dict(size=5), hovertext=np.arange(len(positions)))
est_traj_trace.name = "Estimated"
fig = go.Figure(data=[gt_traj_trace, est_traj_trace])
fig.update_layout(width=1500, height=600, scene=dict(aspectmode='data', zaxis=dict(showticklabels=False, showaxeslabels=False)),
    legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99))
fig.show()

In [ ]:
gt_traj_trace_2d = go.Scatter(x=rover_positions[:,0], y=rover_positions[:,1], mode='lines+markers',
    marker=dict(size=5), hovertext=np.arange(len(rover_positions)))
gt_traj_trace_2d.name = "Ground-truth"
est_traj_trace_2d = go.Scatter(x=positions[:,0], y=positions[:,1], mode='lines+markers',
    marker=dict(size=5), hovertext=np.arange(len(positions)))
est_traj_trace_2d.name = "Estimated"
fig = go.Figure(data=[gt_traj_trace_2d, est_traj_trace_2d])
fig.update_layout(width=1500, height=600, scene=dict(aspectmode='data'),
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()

Generate Map

In [ ]:
# Initialize transformed scans
scans_transformed = num_scans * [None]
for i in range(num_scans):
    scans_transformed[i] = deepcopy(scans[i])
    scans_transformed[i].transform(rover_rotations[:,:,i], rover_positions[i])

In [ ]:
merged = scans_transformed[0]

for s in scans_transformed[1:]:
    merged = merged.merge(s, dist_thresh=0.1)
    #merged.reduce_inside(p2p_dist_thresh=0.1)
    merged.remove_small_planes(area_thresh=0.1)
    merged.fuse_edges(vertex_merge_thresh=0.1)

In [ ]:
# Plot merge
fig = go.Figure(data=merged.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()